In [1]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [11]:
import os
from langchain_community.document_loaders import PyPDFLoader
pdfs = os.listdir('./pdfs')
documents = []
for pdf in pdfs:
    loader = PyPDFLoader(f'./pdfs/{pdf}')
    pages = loader.load_and_split()
    documents.extend(pages)
    
len(documents)

Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong pointing object 109 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 157 0 (offset 0)
Ignoring wrong pointing object 159 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 94 0 (offset 0)
Ignoring wrong pointing object 181 0 (offset 0)
Ignoring

619

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter  # This splitter splits the chunks from the end of the line

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100, add_start_index = True)
chunks = text_splitter.split_documents(documents)
print(len(chunks))

1766


In [2]:
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

#embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
# vectorstore = Chroma.from_documents(small_chunks,embedding=embeddings)
# vectorstore.save_local()

c:\Users\Python\openai\RAG-dev\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vectorstore = FAISS.from_documents(documents=chunks,embedding=embeddings)

NameError: name 'FAISS' is not defined

In [28]:
vectorstore.save_local('vector_db')

In [3]:
vectorstore = FAISS.load_local("vector_db_trans1", embeddings ,allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

## HyDE

In [4]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(model="gpt-3.5-turbo", temperature=0) | StrOutputParser() 
)

# Run
question = "How to release suffering?"
generate_docs_for_retrieval.invoke({"question":question})

'Suffering is a complex and multifaceted phenomenon that can manifest in various forms, including physical, emotional, and psychological distress. In order to effectively release suffering, it is important to understand the underlying causes and mechanisms that contribute to its persistence. \n\nOne approach to releasing suffering is through the practice of mindfulness and meditation. Research has shown that mindfulness-based interventions can help individuals cultivate a greater sense of awareness and acceptance of their present moment experience, which can in turn reduce the intensity of suffering. By learning to observe and acknowledge their thoughts and emotions without judgment, individuals can develop a more compassionate and non-reactive relationship with their suffering, ultimately leading to its release.\n\nAnother effective strategy for releasing suffering is through the use of cognitive-behavioral therapy (CBT). CBT is a psychotherapeutic approach that focuses on identifying

In [5]:
retrieval_chain = generate_docs_for_retrieval | retriever
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(page_content="that show up appr opriately to some matter. And as we think about what we're going to do \nwith the matter either we accept it, do something, change it, our emotions and feelings will \nchange with that. The suffering is when we get stuck in something and usually the stuckness \nis because we're not going to take whatever it is and we'll say it shouldn't be this way and it's \neither my fault because of my character or it's the world's fault because it just wasn't going to \ngive it to me. And that piece of suffering is that stuckness  that we get create.  \nThat's the one that we need to learn to let go to watch the belief system and thought process \nthat is side by side with that and start doing reality check, opening it up, learning the skills of \nbeing with what shows up, being with a n emotion and the feeling that shows up, seeing what \nyou could do about it, do what you need to do and accept the rest and then there's that. That's \nhow the suffering is 

In [6]:
# RAG RAAG
template = """You are an empathetic psychotherapist, parenting counselor, life-coach and mental health expert. 
Be empathetic while answering if the question includes some bad situation. 
Answer the following question based on the context given below, explain each point in a simple language.
Don't look for the answers from other resources. If you don't find the answer from the context, just simply answer: 
"I don't know" or "I didn't understand this, could you please provide some more information on it".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'To release suffering, it is important to understand that suffering arises when we get stuck in a situation and resist accepting it as it is. Here are some steps to help release suffering:\n\n1. Acceptance: Acknowledge the situation for what it is without judgment or resistance. Understand that some things are beyond our control and it is okay to feel the emotions that come with it.\n\n2. Reality check: Challenge any negative beliefs or thoughts that are contributing to your suffering. Question the validity of these beliefs and try to see the situation from a different perspective.\n\n3. Embrace emotions: Allow yourself to feel the emotions that arise naturally in response to the situation. It is okay to feel sadness, grief, or any other emotion that comes up.\n\n4. Take action: Identify what aspects of the situation you can control or change, and take steps to address them. Focus on what you can do rather than getting stuck in what you cannot change.\n\n5. Let go: Practice letting go 

## Decomposition

In [21]:
question = "How to release suffering?"

In [22]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

questions = generate_queries_decomposition.invoke({"question":question})

In [24]:
questions

['1. What are effective techniques for managing emotional pain?',
 '2. How can mindfulness practices help alleviate suffering?',
 '3. Are there specific therapy approaches that focus on releasing suffering?']

In [25]:
# Answer each sub-question individually 

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG prompt
template = """You are an empathetic psychotherapist, parenting counselor, life-coach and mental health expert. 
Be empathetic while answering if the question includes some bad situation. 
Answer the following question based on the context given below, explain each point in a simple language.
Don't look for the answers from other resources. If you don't find the answer from the context, just simply answer: 
"I don't know" or "I didn't understand this, could you please provide some more information on it".

Context: {context}

Question: {question}
"""

prompt_rag = ChatPromptTemplate.from_template(template)
# prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [26]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """

Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'To release suffering, individuals can utilize effective techniques such as gradually and carefully exploring their painful negative core beliefs and accompanying feelings, accepting their emotions, identifying the source of the original pain, and releasing these emotions from their bodies. Mindfulness practices can also be beneficial in alleviating suffering by increasing awareness, changing perspective, promoting acceptance and action, and providing a reality check. Therapies or treatments that target the root causes of suffering, such as guiding clients through their painful beliefs and emotions, can also help individuals release suffering and promote healing. By combining these approaches, individuals can work towards releasing suffering and finding relief from emotional pain.'